[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2/llama-2-13b-retrievalqa.ipynb)

# RAG with LLaMa 13B

In this notebook we'll explore how we can use the open source **Llama-13b-chat** model in both Hugging Face transformers and LangChain.
At the time of writing, you must first request access to Llama 2 models via [this form](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) (access is typically granted within a few hours). If you need guidance on getting access please refer to the beginning of this [article](https://www.pinecone.io/learn/llama-2/) or [video](https://youtu.be/6iHVJyX2e50?t=175).

---

🚨 _Note that running this on CPU is sloooow. If running on Google Colab you can avoid this by going to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**. This should be included within the free tier of Colab._

---

We start by doing a `pip install` of all required libraries.

In [3]:
!pip -q install tensorboard tensorboard-plugin-profile tensorflow
!pip install -U xprof protobuf

  Using cached protobuf-6.33.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
Using cached protobuf-6.33.2-cp39-abi3-manylinux2014_x86_64.whl (323 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.2 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.2 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.2 which is incompatible.


In [1]:
# !pip install transformers torch
!hf auth login # Might show error but the token still grants access.


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `colab_token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentic

# Llama pre trained model

In [2]:
from transformers import pipeline
import torch
from google.colab import userdata
userdata.get('colab_token')

model_id = "meta-llama/Llama-3.2-3B"

pipe = pipeline(
            "text-generation",
            model=model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

pipe("The key to life is")

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "The key to life is to live it.\nWhen you are old and gray and full of sleep, and nodding by the fire, take down this book and slowly read, and dream...\nThe best things in life are the things that you don't pay for.\nThe best things in life are the things that you don't pay for.\nThe best things in life are the things that you don't pay for.\nI have always believed that if one advances confidently in the direction of his dreams, and endeavors to live the life which he has imagined, he will meet with a success unexpected in common hours.\nI have always believed that if one advances confidently in the direction of his dreams, and endeavors to live the life which he has imagined, he will meet with a success unexpected in common hours.\nWe have to accept that people will let us down, and we will let them down. That's part of life.\nWe have to accept that people will let us down, and we will let them down. That's part of life.\nLife is not about finding yourself. Life i

In [ ]:
pipe("The first thought")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [3]:
import os, time, datetime, math
import jax, jax.numpy as jnp
from jax.profiler import start_trace, stop_trace, StepTraceAnnotation

from google.colab import drive
drive.mount('/content/drive')

print("JAX:", jax.__version__)
print("Devices:", jax.devices())

# options = jax.profiler.ProfileOptions()
# options.python_tracer_level = 3 # Python function call tracing
# options.host_tracer_level   = 3 # whether to record accelerator device activity
# options.device_tracer_level = 3 # how much host (CPU) activity to record

# (Optional) set before first jit/compile to dump HLOs for deeper inspection
# os.environ["XLA_FLAGS"] = "--xla_tpu_enable_tracing=true --xla_gpu_tracer_level=2 --xla_dump_to=/content/drive/MyDrive/jax_profile/xla --xla_dump_hlo_as_text"
# JAX_DISABLE_JIT=1

Mounted at /content/drive
JAX: 0.7.2
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)]


In [4]:
# -------- TensorBoard trace --------
logdir = f"/content/drive/MyDrive/jax_profile/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
os.makedirs(logdir, exist_ok=True)

# Attempt to stop any previously running trace to avoid RuntimeError
try:
    jax.profiler.stop_trace()
except RuntimeError:
    pass # No trace was running, or it was already stopped

start_trace(logdir)
# Label a region in the trace for clarity
with StepTraceAnnotation("matmul_bench"):
    pipe("The first thought")
jax.profiler.save_device_memory_profile(logdir + "/memory.prof")
stop_trace()


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [8]:
print(logdir)

/content/drive/MyDrive/jax_profile/20251210-104615


In [ ]:
# %load_ext tensorboard
%tensorboard --load_fast=false --logdir /content/drive/MyDrive/jax_profile/20251210-104615

Launching TensorBoard...